In [8]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import simpleaudio as sa
from  time import time

In [9]:
model=tf.keras.models.load_model("emotion_classification_model")

In [10]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [11]:
dictionnaire={0:'angry',1:'disgust',2:'fear',3:'happy',4:'neutral',5:'sad',6:'surprise'}

In [13]:
cap= cv2.VideoCapture(0)

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer= cv2.VideoWriter('basicvideo.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))
wave_obj = sa.WaveObject.from_wave_file("ressources/triste.wav")
timeStart=time()

imagePathDict={0:"ressources/angry.png",1:"ressources/disgust.png",2:"ressources/fear.png",3:"ressources/happy.png",4:"ressources/neutral.png",5:"ressources/sad.png",6:"ressources/surprise.png"}

i=0
while True:
    ret,frame= cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray=cv2.resize(roi_gray,(48,48))
        x=img_to_array(roi_gray)
        idx=np.argmax(model.predict(np.array(x).reshape(-1,48,48,1)))
        image = cv2.putText(frame, dictionnaire[idx],(50, 50),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2,cv2.LINE_AA)


        s_img = cv2.imread(imagePathDict[idx])
        s_img=cv2.resize(s_img,(50,50))

        l_img = frame
        x_offset=y_offset=50
        frame[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img

        timeStop=time()
        if timeStop-timeStart >15 and idx==5 :
            timeStart=timeStop
            play_obj = wave_obj.play()

    

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord("q"):
        break


cap.release()
writer.release()
cv2.destroyAllWindows()